In [2]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Mounted at /content/drive
/content/drive/My Drive


In [3]:
!git clone https://github.com/xelav/Insider-Threat-CERT /root/repo

import sys
sys.path.append("/root/repo")

Cloning into '/root/repo'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 145 (delta 93), reused 111 (delta 65), pack-reused 0
Receiving objects: 100% (145/145), 109.35 KiB | 678.00 KiB/s, done.
Resolving deltas: 100% (93/93), done.


In [4]:
!pip install pytorch-ignite

     |████████████████████████████████| 112kB 8.9MB/s 


In [47]:
%cd "/root/repo"
!git pull
%cd "{root_dir}"

/root/repo
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/xelav/Insider-Threat-CERT
   598bf49..60fdac9  master     -> origin/master
Updating 598bf49..60fdac9
Fast-forward
 src/lstm_trainer.py | 33 +++++++++++++++++++--------------
 1 file changed, 19 insertions(+), 14 deletions(-)
/content/drive/My Drive


In [39]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm.notebook import tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.lstm_trainer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO:

* Больше детерминизма



In [0]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(f'{root_dir}/Datasets/CERT_output')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

run_name = 'lstm/base5'

log_dir = output_dir / 'logs' / run_name
checkpoint_dir = output_dir / 'checkpoints' / run_name

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

if log_dir.is_dir():
    shutil.rmtree(log_dir)
if checkpoint_dir.is_dir():
    shutil.rmtree(checkpoint_dir)

In [0]:
params = get_params()
params['model']['lstm_encoder']['embedding_size'] = None

# Train

TODO:
* Эксперименты с LR Scheduling

In [0]:
actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl', main_answers_file, min_length=50, max_length=200)
cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=1024)

device = 'cuda'

In [0]:
lstm_encoder = LSTM_Encoder(params['model']['lstm_encoder'])
criterion = nn.NLLLoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        log_dir=log_dir.as_posix(),
                                        checkpoint_dir=checkpoint_dir,
                                        tensorboard_every=10,
                                       )

val_engine = create_supervised_evaluator_lstm(
        lstm_encoder, device=device,
        prepare_batch=prepare_batch_lstm,
        metrics={},
        criterion=criterion,
        log_dir=log_dir.as_posix(),
        checkpoint_dir=checkpoint_dir,
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    print('Validation run:')
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)


In [42]:
train_engine.run(train_loader, max_epochs=50)

Initial validation run:



Validation Results - Avg loss: 4.063937, Accuracy: 0.002377, Non-Pad-Accuracy: 0.017508



Epoch results - Avg loss: 1.921195, Accuracy: 0.738630, Non-Pad-Accuracy: 0.282846
Validation run:



Validation Results - Avg loss: 0.605426, Accuracy: 0.885443, Non-Pad-Accuracy: 0.641481



Epoch results - Avg loss: 0.570747, Accuracy: 0.885207, Non-Pad-Accuracy: 0.643744
Validation run:


...


Validation Results - Avg loss: 0.229682, Accuracy: 0.954209, Non-Pad-Accuracy: 0.646633



Epoch results - Avg loss: 0.218651, Accuracy: 0.955623, Non-Pad-Accuracy: 0.647714
Validation run:



Validation Results - Avg loss: 0.211212, Accuracy: 0.956714, Non-Pad-Accuracy: 0.646216


Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: ignored

In [32]:
%ls "{checkpoint_dir}"

'best_model_23_val_loss=0.7341190824534809.pth'   checkpoint_1748.pth
 checkpoint_1672.pth                              checkpoint_1824.pth


In [0]:
from google.colab import files
files.download(checkpoint_dir / 'final_model_760.pth') 

In [0]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

# Prediction exploration

TODO:
- decode class numbers

In [0]:
batch = next(iter(val_loader))

In [0]:
x, y = prepare_batch_lstm(batch)

In [0]:
lstm_encoder.train()

In [0]:
y.argmax(dim=2)[2]

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2)[2]

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2).unique()